# Lecture 1: The Fourier Expansion and Orthogonality of Characters

**Based on CS294-92: Analysis of Boolean Functions (Spring 2025)**  
**Instructor: Avishay Tal**  
**Notes by: Joyce Lu**

This notebook follows along with Lecture 1, demonstrating the concepts using the `boolfunc` library.

---

## Learning Objectives

1. Understand what a Boolean function is and how to represent it
2. Learn the Fourier expansion of Boolean functions as multilinear polynomials
3. Understand orthogonality of characters (Walsh functions)
4. Apply Parseval's and Plancherel's identities
5. Use the inversion formula to compute Fourier coefficients

In [ ]:
import numpy as np
import boolfunc as bf
from boolfunc.analysis.fourier import (
    parseval_verify, 
    plancherel_inner_product,
    fourier_degree,
    spectral_norm
)

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

## 1.1 What is a Boolean Function?

A **Boolean function** is defined as $f: \{0,1\}^n \to \{0,1\}$.

Boolean functions can model:
- **Pseudorandomness**: A test we want to fool
- **Combinatorics**: Set systems via indicator functions  
- **Social choice**: $n$ votes aggregated into a decision
- **Coding theory**: Error-correcting codes
- **Graph properties**: Properties of graphs encoded in binary

In [ ]:
# Creating Boolean functions with boolfunc

# From a truth table
xor_2 = bf.create([0, 1, 1, 0])  # XOR of 2 variables
print("XOR function created from truth table:")
print(f"  XOR(0,0) = {xor_2.evaluate(0)}")
print(f"  XOR(0,1) = {xor_2.evaluate(1)}")
print(f"  XOR(1,0) = {xor_2.evaluate(2)}")
print(f"  XOR(1,1) = {xor_2.evaluate(3)}")

# Using built-in functions
and_3 = bf.AND(3)
or_3 = bf.OR(3)
maj_3 = bf.majority(3)
parity_3 = bf.parity(3)

print("\nBuilt-in Boolean functions (n=3):")
print(f"  AND(1,1,1) = {and_3.evaluate(7)}")
print(f"  OR(0,0,1) = {or_3.evaluate(1)}")
print(f"  MAJORITY(1,1,0) = {maj_3.evaluate(6)}")
print(f"  PARITY(1,0,1) = {parity_3.evaluate(5)}")

## 1.2 The Fourier Expansion

**Theorem 1.3 (Fundamental Theorem of Boolean Functions):**

Every Boolean function $f: \{\pm 1\}^n \to \mathbb{R}$ can be uniquely represented as a multilinear polynomial:

$$f(x) = \sum_{S \subseteq [n]} \hat{f}(S) \cdot \chi_S(x)$$

where:
- $[n] = \{1, \ldots, n\}$
- $\chi_S(x) = \prod_{i \in S} x_i$ is a **character** (Walsh function)
- $\hat{f}(S)$ is the **Fourier coefficient** of $f$ on $S$

### Example: The Majority Function

$$\text{maj}_3(x_1, x_2, x_3) = \frac{1}{2}x_1 + \frac{1}{2}x_2 + \frac{1}{2}x_3 - \frac{1}{2}x_1 x_2 x_3$$

In [ ]:
# Computing the Fourier expansion of MAJORITY₃
maj_3 = bf.majority(3)
# Direct API: f.fourier()
fourier = maj_3.fourier()

print("Fourier expansion of MAJORITY₃:")
for s in range(8):
    if abs(fourier[s]) > 1e-10:
        # Convert index to set notation
        bits = [i+1 for i in range(3) if (s >> i) & 1]
        set_str = "{" + ",".join(map(str, bits)) + "}" if bits else "∅"
        print(f"  f̂({set_str:6}) = {fourier[s]:+.4f}")

print("\nSo: maj₃(x) = (1/2)x₁ + (1/2)x₂ + (1/2)x₃ - (1/2)x₁x₂x₃")
print("  ✓ Matches the theoretical result!")

## 1.3 Parseval's Identity

**Corollary 1.7 (Parseval's Identity):** For any $f: \{\pm 1\}^n \to \{\pm 1\}$:

$$\sum_{S \subseteq [n]} \hat{f}(S)^2 = 1$$

This is because $\langle f, f \rangle = \mathbb{E}[f(x)^2] = \mathbb{E}[1] = 1$ when $f$ takes values in $\{\pm 1\}$.

In [ ]:
# Verify Parseval's identity for several functions
functions = {
    "AND₃": bf.AND(3),
    "OR₃": bf.OR(3),
    "MAJORITY₃": bf.majority(3),
    "PARITY₃": bf.parity(3),
}

print("Parseval's Identity: Σ f̂(S)² = 1 for Boolean functions\n")
print(f"{'Function':<12} | {'Σ f̂(S)²':<10} | Parseval")
print("-" * 40)

for name, f in functions.items():
    # Direct API: f.fourier()
    fourier = f.fourier()
    sum_squares = sum(c**2 for c in fourier)
    verified = parseval_verify(f)
    status = "✓" if verified else "✗"
    print(f"{name:<12} | {sum_squares:<10.6f} | {status}")

## 1.4 Plancherel's Identity

**Corollary 1.6 (Plancherel's Identity):** For any $f, g: \{\pm 1\}^n \to \mathbb{R}$:

$$\langle f, g \rangle = \sum_{S \subseteq [n]} \hat{f}(S) \hat{g}(S)$$

The inner product in the "time domain" equals the inner product in the "frequency domain".

In [ ]:
# Verify Plancherel's identity
f = bf.AND(3)
g = bf.OR(3)

# Compute inner product directly: ⟨f, g⟩ = E[f(x)g(x)]
n = 3
direct_ip = 0
for x in range(1 << n):
    fx = 1 - 2 * f.evaluate(x)  # ±1
    gx = 1 - 2 * g.evaluate(x)  # ±1
    direct_ip += fx * gx
direct_ip /= (1 << n)

# Compute via Fourier domain
fourier_ip = plancherel_inner_product(f, g)

print("Plancherel's Identity verification:")
print(f"  Direct inner product ⟨AND, OR⟩ = {direct_ip:.6f}")
print(f"  Fourier inner product Σ f̂(S)ĝ(S) = {fourier_ip:.6f}")
print(f"  Match: {'✓' if abs(direct_ip - fourier_ip) < 1e-10 else '✗'}")

## 1.5 Influences and Spectral Properties

The Fourier expansion reveals structural properties:

- **Influence of variable $i$**: $\text{Inf}_i(f) = \sum_{S \ni i} \hat{f}(S)^2$
- **Total influence**: $\text{Inf}(f) = \sum_i \text{Inf}_i(f) = \sum_S |S| \cdot \hat{f}(S)^2$

In [ ]:
# Analyze spectral properties of MAJORITY₅
f = bf.majority(5)

print("Spectral Analysis of MAJORITY₅:")
print("=" * 40)

# Direct API: f.influences() and f.total_influence()
influences = f.influences()
total_inf = f.total_influence()

print(f"\nVariable influences:")
for i, inf in enumerate(influences):
    print(f"  Inf_{i+1}(f) = {inf:.4f}")

print(f"\nTotal influence: {total_inf:.4f}")

# Degree
deg = fourier_degree(f)
print(f"Fourier degree: {deg}")

## Summary

### Key Takeaways from Lecture 1:

1. **Boolean functions** $f: \{\pm 1\}^n \to \{\pm 1\}$ model many computational problems

2. **Fourier expansion**: Every Boolean function is a unique multilinear polynomial
   $$f(x) = \sum_{S \subseteq [n]} \hat{f}(S) \chi_S(x)$$

3. **Characters** $\chi_S(x) = \prod_{i \in S} x_i$ form an orthonormal basis

4. **Inversion formula**: $\hat{f}(S) = \langle f, \chi_S \rangle = \mathbb{E}[f(x) \chi_S(x)]$

5. **Plancherel**: $\langle f, g \rangle = \sum_S \hat{f}(S) \hat{g}(S)$

6. **Parseval**: $\sum_S \hat{f}(S)^2 = 1$ for Boolean-valued functions

### Next: Property Testing (Lecture 2)
- The BLR linearity test
- If $f(x+y) \approx f(x) + f(y)$ for most inputs, is $f$ close to linear?

---

*This notebook was created for CS294-92 using the `boolfunc` library.*